<a href="https://colab.research.google.com/github/aida-wofford/Servicio_IAUNAM/blob/main/Journal_Rankings_and_Author_Position.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
__author__ = 'Aida Wofford <awofford@astro.unam.mx>, Timothy Wofford <wofford.timothy@gmail.com>'
__version__ = '20210623' # fecha de creación
__datasets__ = ['ADS','SRJ']  # datos usados en este notebook
__keywords__ = ['ADS', 'SRJ', 'IA UNAM', 'Informe Anual'], # palabras clave relevantes a este notebook

# Instructions

## Step 1: Get the API Token for ADS

* Register an account on the ADS website if you don't have one.
* Login to the ADS website: https://ui.adsabs.harvard.edu/user/account/login
* In the upper-right corner, go to Account > Settings
* In the left menu, go to API Token
* Generate a new key
* Save the key as a string in the <a href="#Environment-setup-and-Input-parameters">Environment setup</a> section below.

## Step 2: Download journal rankings

* To get the journal ranking data go to: https://www.scimagojr.com/journalrank.php?category=3103&openaccess=false
* Download the data by clicking onto the "Download data" button on the right side, just above the list.
  * Although the file is saved as a csv, I had to open it in Numbers and export it to csv in order for pandas to import it correctly into python.
  * I also had to make sure the filename did not include spaces.
* Save the filename in the variable <code>journal_rankings_file</code> in the <a href="#Input-Parameters">Input Parameters</a> section below.

The code below assumes that the journal title is under the 'Title' column and that the ranking is under the 'SJR Quartile' column.

## Step 3: Get lists of author names

* We will ask people to contribute to a google spreadsheet where they will enter the list of exact names that they used in publications this year. 

* The first name in the list provided by the author should be complete to avoid including different authors who share the same initials.

Example:
<pre><code>authors=[
    ["Kirichenko, Aida Yu.", "Kirichenko, A."],
    ["Wofford, Aida", "Wofford, A."]
]</code></pre>


# Environment setup and Input parameters

In [9]:
# If you don't already have the ADS API module installed, uncomment the next line.

#!pip install ads

import os
import ads
import pandas as pd

# this is your personal API Token from ADS 
os.environ['ADS_DEV_KEY'] = ''#'paste your key here'

# file containing journal ranking data
journal_rankings_file='https://raw.githubusercontent.com/aida-wofford/Servicio_IAUNAM/main/scimagojr2020SubjectCategory-AstronomyandAstrophysics.csv'

# Each author should provide a list of names as they appear in publications.
# The first name in the list provided by the author should be complete
# to avoid including different authors who share the same initials.
authors=[
    ["Kirichenko, Aida Yu.", "Kirichenko, A."],
    ["Wofford, Aida", "Wofford, A."]
]


# Program

You shouldn't have to edit anything in this section.

In [10]:
def verificacion_de_produccion(author):
    # load the journal ranking data
    journal_ranking = pd.read_csv(journal_rankings_file)

    # query the ADS database for the author's bibliographic data
    papers = list(ads.SearchQuery(author=author[0], year="2021", property='refereed'))

    # Some journal titles mistakenly begin with "The ". Remove the offending word.
    # Verify that the author is in the author list
    # and whether it is in the top10authors list
    # and whether it is in the top2authors list
    author_data = [[
        paper.pub.replace('The ', ''),
        any(name in author for name in paper.author),
        any(name in author for name in paper.author[0:10]),
        any(name in author for name in paper.author[0:2])
        ] for paper in papers]
    
    # look up the journal ranking for each publication; keep other columns
    author_data = [ [journal_ranking.loc[journal_ranking['Title']== publication[0] ]['SJR Quartile'].iloc[0], publication[0], publication[1], publication[2], publication[3]] for publication in author_data]
    
    # convert data to dataframe for further manipulations
    output = pd.DataFrame(author_data,columns=['Ranking','Journal','Author','Top10','Top2'])
    return [author, output, papers]

# Output

In [11]:
results = [verificacion_de_produccion(author) for author in authors]

/usr/local/lib/python3.7/dist-packages/ads/utils.py:31: UserWarning: You are lazy loading attributes via 'pub', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


In [12]:
from IPython.display import display, HTML
for result in results:
    print(result[0])
    display(HTML(result[1].to_html()))
    print('----------------------------------------------------------------\n\n')

['Kirichenko, Aida Yu.', 'Kirichenko, A.']


,Ranking,Journal,Author,Top10,Top2
0,Q1,Astrophysical Journal,True,True,False
1,Q1,Monthly Notices of the Royal Astronomical Society,True,True,False


----------------------------------------------------------------


['Wofford, Aida', 'Wofford, A.']


,Ranking,Journal,Author,Top10,Top2
0,Q1,Monthly Notices of the Royal Astronomical Society,True,True,True
1,Q1,Astronomy and Astrophysics,True,False,False
2,Q1,Astrophysical Journal,True,False,False


----------------------------------------------------------------


